In [7]:
from langchain_community.chat_models import ChatOllama
from openai import OpenAI
import keyring

llm_llama = ChatOllama(model='llama3.1')
llm_openai = OpenAI(
    api_key=keyring.get_password('openai', 'key_for_windows')
)

def get_response_from_llama(prompt):
    response = llm_llama.invoke(prompt)
    return response.content

def get_response_from_openai(prompt):
    completion = llm_openai.chat.completions.create(
        messages=[
            {'role':'user', 'content':prompt}
        ],
        model='gpt-3.5-turbo',
        max_tokens=1024,
        temperature=0.0
    )
    return completion.choices[0].message.content

In [8]:
document = "AI메모리반도체 수요가 증가하는 가운데, SK하이닉스와 삼성전자가 올해 본격적으로 HBM 반도체 생산에 열을 올릴 예정입니다. 정민수 연구원은 HBM에 대해서 새로운 아키텍쳐를 설계하여 기존 대비 약 30%의 효율 향상을 이끌었다고 소개하고 있습니다."
prompt = f"""주어진 텍스트에 대해서 코스피에 상장된 기업을 추출하세요.
###
{document}"""

answer = get_response_from_llama(prompt)
print(answer)

코스피에 상장된 기업은 두 개이며, 그들은 SK하이닉스와 삼성전자입니다.


In [9]:
prompt = f"""주어진 텍스트에 대해서 코스피에 상장된 기업을 추출하세요.
다음과 같이 json 포맷으로 출력하세요.
{{기업명:종목코드,...}}
###
{document}"""

answer = get_response_from_llama(prompt)
print(answer)

코스피에 상장된 기업인 SK하이닉스는 '096970' 코드로, 삼성전자는 '005930' 코드를 가지고 있습니다.

따라서, json 포맷으로 출력하기 위해 다음과 같이 구성할 수 있습니다:

```
{
  "SK하이닉스": "096970",
  "삼성전자": "005930"
}
```


In [14]:
# 학습 이후의 데이터에 대해서 물어보는 경우
# (1) 지식베이스를 프롬프트에 넣어줌

prompt = f"""주어진 텍스트에 대해서 코스피에 상장된 기업을 추출하세요.
지식베이스는 기업명:종목코드 형식으로 구성되어 있습니다.
다음과 같이 json 포맷으로 출력하세요.
{{기업명:종목코드,...}}
반드시 텍스트에 포함되고, 지식베이스에 포함된 기업명만을 추출합니다.

# 지식베이스
SK하이닉스 : 0001
삼성바이오로직스 : 0002
현대농기계 : 0003

# 텍스트:
{document}"""

answer = get_response_from_llama(prompt)
print(answer)

아래와 같이 json 포맷으로 코스피에 상장된 기업을 추출했습니다.

```json
{
  "SK하이닉스": "0001",
  "삼성전자": "005930"
}
```

해석: 텍스트와 지식베이스에 등장하는 '삼성전자는'이라는 문구에서 '삼성'은 '삼성전자'로 추정하고, 관련 종목코드를 추가하였습니다.
